In [14]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
import json
from folium.features import GeoJsonTooltip
import branca
from folium.plugins import TagFilterButton
from folium.plugins import TimeSliderChoropleth
from folium.plugins import Draw
from folium.plugins import GroupedLayerControl
from folium.plugins import Search
from branca.colormap import linear
import copy

In [15]:
levl0=gpd.read_file("europe.geojson")
levl2=gpd.read_file("NUTS_2_Q2.geojson")
levl1=gpd.read_file("NUTS_1_Q1.geojson")
levl3=gpd.read_file("NUTS_3_Q1.geojson")

shipement_profil=pd.read_excel("MATRICE shipment profile.xlsx")
zip_code=pd.read_excel("zipcode_nuts with uk instead of gb.xlsx")
zip_code_for_tag=pd.read_excel("zipcodefortag.xlsx")
zip_code_for_tag2=pd.read_excel("zip_code for tage 2.xlsx")
zip_code_for_tag1=pd.read_excel("zip_code for nuts1.xlsx")
zip_code_for_tag0=pd.read_excel("zip code for tag 0.xlsx")
dsv=pd.read_excel("DSV Branches.xlsx")

In [17]:
shipement_profil.loc[shipement_profil['ZC from'].str[:2] == 'AD', 'ZC from'] = 'AD'
shipement_profil.loc[shipement_profil['ZC to'].str[:2] == 'AD', 'ZC to'] = 'AD'

shipement_profil.loc[shipement_profil['ZC from'].str[:2] == 'ME', 'ZC from'] = 'ME'
shipement_profil.loc[shipement_profil['ZC to'].str[:2] == 'ME', 'ZC to'] = 'ME'

shipement_profil.loc[shipement_profil['ZC from'].str[:2] == 'RU', 'ZC from'] = 'RU'
shipement_profil.loc[shipement_profil['ZC to'].str[:2] == 'RU', 'ZC to'] = 'RU'

shipement_profil.loc[shipement_profil['ZC from'].str[:2] == 'XK', 'ZC from'] = 'XK'
shipement_profil.loc[shipement_profil['ZC to'].str[:2] == 'XK', 'ZC to'] = 'XK'

shipement_profil.loc[shipement_profil['ZC from'].str[:2] == 'BA', 'ZC from'] = 'BA'
shipement_profil.loc[shipement_profil['ZC to'].str[:2] == 'BA', 'ZC to'] = 'BA'

In [18]:
levl3.drop_duplicates(subset="NUTS_ID", keep="first", inplace=True)

In [19]:
ZC_from=shipement_profil["ZC from"].unique().tolist()

df={}
for country_from in ZC_from:
    df_=shipement_profil[shipement_profil["ZC from"]==country_from]
    df[country_from]=df_

In [20]:
GRP={}
LTL={}
FTL={}
for key,value in df.items():

    GRP_=df[key][df[key]["Product"]=="GRP"]
    LTL_=df[key][df[key]["Product"]=="LTL"]
    FTL_=df[key][df[key]["Product"]=="FTL"]
    GRP[key]=GRP_
    LTL[key]=LTL_
    FTL[key]=FTL_

In [21]:
for key,value in df.items():
    df[key]=df[key].groupby(["ZC to"],as_index=False)["PW DSV"].sum()


In [22]:
for key,value in GRP.items():
    GRP[key]=GRP[key].groupby(["ZC to"],as_index=False)["PW DSV"].sum()

In [23]:
for key,value in LTL.items():
    LTL[key]=LTL[key].groupby(["ZC to"],as_index=False)["PW DSV"].sum()

In [24]:
for key,value in FTL.items():
    FTL[key]=FTL[key].groupby(["ZC to"],as_index=False)["PW DSV"].sum()

In [25]:
for key,value in df.items():
    df[key]=pd.merge(df[key],zip_code,on='ZC to',how="left")



In [26]:
for key,value in GRP.items():
    GRP[key]=pd.merge(GRP[key],zip_code,on='ZC to',how="left")

In [27]:
for key,value in LTL.items():
    LTL[key]=pd.merge(LTL[key],zip_code,on='ZC to',how="left")

In [28]:
for key,value in FTL.items():
    FTL[key]=pd.merge(FTL[key],zip_code,on='ZC to',how="left")

In [29]:
for key,value in df.items():
    df[key]['count'] = df[key].groupby('ZC to')['ZC to'].transform('count')



In [30]:
for key,value in GRP.items():
    GRP[key]['count'] = GRP[key].groupby('ZC to')['ZC to'].transform('count')

In [31]:
for key,value in LTL.items():
    LTL[key]['count'] = LTL[key].groupby('ZC to')['ZC to'].transform('count')

In [32]:
for key,value in FTL.items():
    FTL[key]['count'] = FTL[key].groupby('ZC to')['ZC to'].transform('count')

In [33]:
for key,value in df.items():
    df[key]["PW DSV"]=(df[key]["PW DSV"])/df[key]["count"]

In [34]:
for key,value in GRP.items():
    GRP[key]["PW DSV"]=(GRP[key]["PW DSV"])/GRP[key]["count"]

In [35]:
for key,value in FTL.items():
    FTL[key]["PW DSV"]=(FTL[key]["PW DSV"])/FTL[key]["count"]

In [36]:
for key,value in LTL.items():
    LTL[key]["PW DSV"]=(LTL[key]["PW DSV"])/LTL[key]["count"]

In [37]:
df3=copy.deepcopy(df)
df2=copy.deepcopy(df)
df1=copy.deepcopy(df)
df0=copy.deepcopy(df)


In [38]:
GRP3=copy.deepcopy(GRP)
GRP2=copy.deepcopy(GRP)
GRP1=copy.deepcopy(GRP)
GRP0=copy.deepcopy(GRP)

In [39]:
LTL3=copy.deepcopy(LTL)
LTL2=copy.deepcopy(LTL)
LTL1=copy.deepcopy(LTL)
LTL0=copy.deepcopy(LTL)

In [40]:
FTL3=copy.deepcopy(FTL)
FTL2=copy.deepcopy(FTL)
FTL1=copy.deepcopy(FTL)
FTL0=copy.deepcopy(FTL)

In [41]:
for key,value in df3.items():
    df3[key]=df3[key].groupby(["NUTS3"],as_index=False)["PW DSV"].sum()



In [42]:
for key,value in GRP3.items():
    GRP3[key]=GRP3[key].groupby(["NUTS3"],as_index=False)["PW DSV"].sum()

In [43]:
for key,value in LTL3.items():
    LTL3[key]=LTL3[key].groupby(["NUTS3"],as_index=False)["PW DSV"].sum()

In [44]:
for key,value in FTL3.items():
    FTL3[key]=FTL3[key].groupby(["NUTS3"],as_index=False)["PW DSV"].sum()

In [45]:
for key,value in df3.items():
    df3[key]=pd.merge(df3[key],zip_code_for_tag,on='NUTS3',how="left")

In [46]:
for key,value in GRP3.items():
    GRP3[key]=pd.merge(GRP3[key],zip_code_for_tag,on='NUTS3',how="left")

In [47]:
for key,value in LTL3.items():
    LTL3[key]=pd.merge(LTL3[key],zip_code_for_tag,on='NUTS3',how="left")

In [48]:
for key,value in FTL3.items():
    FTL3[key]=pd.merge(FTL3[key],zip_code_for_tag,on='NUTS3',how="left")

In [49]:
for key,value in df2.items():
    df2[key]=df2[key].groupby(["nuts2"],as_index=False)["PW DSV"].sum()



In [50]:
for key,value in GRP2.items():
    GRP2[key]=GRP2[key].groupby(["nuts2"],as_index=False)["PW DSV"].sum()

In [51]:
for key,value in LTL2.items():
    LTL2[key]=LTL2[key].groupby(["nuts2"],as_index=False)["PW DSV"].sum()

In [52]:
for key,value in FTL2.items():
    FTL2[key]=FTL2[key].groupby(["nuts2"],as_index=False)["PW DSV"].sum()

In [53]:
for key,value in df2.items():
    df2[key]=pd.merge(df2[key],zip_code_for_tag2,on='nuts2',how="left")

In [54]:
for key,value in GRP2.items():
    GRP2[key]=pd.merge(GRP2[key],zip_code_for_tag2,on='nuts2',how="left")

In [55]:
for key,value in LTL2.items():
    LTL2[key]=pd.merge(LTL2[key],zip_code_for_tag2,on='nuts2',how="left")

In [56]:
for key,value in FTL2.items():
    FTL2[key]=pd.merge(FTL2[key],zip_code_for_tag2,on='nuts2',how="left")

In [57]:
for key,value in df1.items():
    df1[key]=df1[key].groupby(["nuts1"],as_index=False)["PW DSV"].sum()



In [58]:
for key,value in GRP1.items():
    GRP1[key]=GRP1[key].groupby(["nuts1"],as_index=False)["PW DSV"].sum()

In [59]:
for key,value in LTL1.items():
    LTL1[key]=LTL1[key].groupby(["nuts1"],as_index=False)["PW DSV"].sum()

In [60]:
for key,value in FTL1.items():
    FTL1[key]=FTL1[key].groupby(["nuts1"],as_index=False)["PW DSV"].sum()

In [61]:
for key,value in df1.items():
    df1[key]=pd.merge(df1[key],zip_code_for_tag1,on='nuts1',how="left")

In [62]:
for key,value in GRP1.items():
    GRP1[key]=pd.merge(GRP1[key],zip_code_for_tag1,on='nuts1',how="left")

In [63]:
for key,value in LTL1.items():
    LTL1[key]=pd.merge(LTL1[key],zip_code_for_tag1,on='nuts1',how="left")

In [64]:
for key,value in FTL1.items():
    FTL1[key]=pd.merge(FTL1[key],zip_code_for_tag1,on='nuts1',how="left")

In [65]:
for key,value in df0.items():
    df0[key]=df0[key].groupby(["nuts0"],as_index=False)["PW DSV"].sum()

In [66]:
for key,value in GRP0.items():
    GRP0[key]=GRP0[key].groupby(["nuts0"],as_index=False)["PW DSV"].sum()

In [67]:
for key,value in LTL0.items():
    LTL0[key]=LTL0[key].groupby(["nuts0"],as_index=False)["PW DSV"].sum()

In [68]:
for key,value in FTL0.items():
    FTL0[key]=FTL0[key].groupby(["nuts0"],as_index=False)["PW DSV"].sum()

In [69]:
GRP0={key: value for key, value in GRP0.items() if not value.empty}
GRP1={key: value for key, value in GRP1.items() if not value.empty}
GRP2={key: value for key, value in GRP2.items() if not value.empty}
GRP3={key: value for key, value in GRP3.items() if not value.empty}

In [70]:
LTL0= {key: value for key, value in LTL0.items() if not value.empty}
LTL1= {key: value for key, value in LTL1.items() if not value.empty}
LTL2= {key: value for key, value in LTL2.items() if not value.empty}
LTL3= {key: value for key, value in LTL3.items() if not value.empty}

In [71]:
FTL0= {key: value for key, value in FTL0.items() if not value.empty}
FTL1= {key: value for key, value in FTL1.items() if not value.empty}
FTL2= {key: value for key, value in FTL2.items() if not value.empty}
FTL3= {key: value for key, value in FTL3.items() if not value.empty}

In [72]:
for key,value in df.items():

    

    m= folium.Map(location=[54.5260,15.2551],zoom_start=4)

    fg0 = folium.FeatureGroup(name='Nuts0',overlay=True,show=False).add_to(m)
    fg1 = folium.FeatureGroup(name='levl1',overlay=True,show=False).add_to(m)
    fg2= folium.FeatureGroup(name='levl2',overlay=True,show=False).add_to(m)
    fg3= folium.FeatureGroup(name='levl3',overlay=True,show=False).add_to(m)
    fg4= folium.FeatureGroup(name="Locations",overlay=True,show=False).add_to(m)
    fg5 = folium.FeatureGroup(name='levl3',overlay=True,show=False).add_to(m)
    fg6 = folium.FeatureGroup(name='levl2',overlay=False,show=False).add_to(m)
    fg7= folium.FeatureGroup(name='levl1',overlay=False,show=False).add_to(m)
    fg8= folium.FeatureGroup(name='Nuts0',overlay=False,show=False).add_to(m)
    fg9 = folium.FeatureGroup(name='levl3',overlay=False,show=False).add_to(m)
    fg10 = folium.FeatureGroup(name='levl2',overlay=False,show=False).add_to(m)
    fg11= folium.FeatureGroup(name='levl1',overlay=False,show=False).add_to(m)
    fg12= folium.FeatureGroup(name='Nuts0',overlay=False,show=False).add_to(m)
    fg13 = folium.FeatureGroup(name='levl3',overlay=False,show=False).add_to(m)
    fg14 = folium.FeatureGroup(name='levl2',overlay=False,show=False).add_to(m)
    fg15= folium.FeatureGroup(name='levl1',overlay=False,show=False).add_to(m)
    fg16= folium.FeatureGroup(name='Nuts0',overlay=False,show=False).add_to(m)

    #all the data
    
    levl3_merge=pd.merge(levl3,df3[key],right_on="NUTS3" ,left_on="NUTS_ID",how="right")
    
    choropleth3=folium.Choropleth(
    geo_data=levl3_merge,
    name="choropleth",
    data=levl3_merge,
    columns=["NUTS3","PW DSV"],
    key_on="properties.NUTS_ID",
    fill_color='OrRd',
    fill_opacity=0.7,
    legend=True,
    highlight=True,
    
    
    
                                 
    ).geojson.add_to(fg3)



    folium.features.GeoJson(
                    data=levl3_merge,
                    name='test',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=["NUTS_NAME","ZC to","PW DSV"],
                                
                               
                        aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth3) 


    levl2_merge=pd.merge(levl2,df2[key],right_on="nuts2" ,left_on="NUTS_ID",how="right")

    choropleth2=folium.Choropleth(
    geo_data=levl2_merge,
    name="choropleth",
    data=levl2_merge,
    columns=["nuts2","PW DSV"],
    key_on="properties.NUTS_ID",
    fill_color='OrRd',
    fill_opacity=0.7,
    legend=True,
    highlight=True,
    
    
    
                                 
    ).geojson.add_to(fg2)



    folium.features.GeoJson(
                    data=levl2_merge,
                    name='test',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=["NUTS_NAME","ZC to","PW DSV"],    
                        aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth2) 

    # #######
    
    levl1_merge=pd.merge(levl1,df1[key],right_on="nuts1" ,left_on="NUTS_ID",how="right")

    choropleth1=folium.Choropleth(
    geo_data=levl1_merge,
    name="choropleth",
    data=levl1_merge,
    columns=["nuts1","PW DSV"],
    key_on="properties.NUTS_ID",
    fill_color='OrRd',
    fill_opacity=0.7,
    legend=True,
    highlight=True,
    
    
    
                                 
    ).geojson.add_to(fg1)



    folium.features.GeoJson(
                    data=levl1_merge,
                    name='test',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=["NUTS_NAME","ZC to","PW DSV"],       
                        aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth1) 


    # ##########
    
    levl0_merge=pd.merge(levl0,df0[key],right_on="nuts0" ,left_on="ISO2",how="right")

    choropleth0=folium.Choropleth(
    geo_data=levl0_merge,
    name="choropleth",
    data=levl0_merge,
    columns=["nuts0","PW DSV"],
    key_on="properties.ISO2",
    fill_color='OrRd',
    fill_opacity=0.7,
    legend=True,
    highlight=True,
    
    
    
                                 
    ).geojson.add_to(fg0)



    folium.features.GeoJson(
                    data=levl0_merge,
                    name='test',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=["NAME","PW DSV"],
                                
                               
                        aliases=["To : ", "Pay Weight : "], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(choropleth0) 
    


    #################################################################################################################################
    #GRP olny:

    # for i in GRP_ZC_from:
    if key in GRP3:
       
        GRP_levl3_merge=pd.merge(levl3,GRP3[key],right_on="NUTS3" ,left_on="NUTS_ID",how="right")
    
        choropleth5=folium.Choropleth(
        geo_data=GRP_levl3_merge,
        name="choropleth",
        data=GRP_levl3_merge,
        columns=["NUTS3","PW DSV"],
        key_on="properties.NUTS_ID",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
        
        
        
                                    
        ).geojson.add_to(fg5)



        folium.features.GeoJson(
                        data=GRP_levl3_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],
                                    
                                
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth5) 
        ########################

    if key in GRP2:

        GRP_levl2_merge=pd.merge(levl2,GRP2[key],right_on="nuts2" ,left_on="NUTS_ID",how="right")

        choropleth6=folium.Choropleth(
        geo_data=GRP_levl2_merge,
        name="choropleth",
        data=GRP_levl2_merge,
        columns=["nuts2","PW DSV"],
        key_on="properties.NUTS_ID",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
        
        
        
                                    
        ).geojson.add_to(fg6)



        folium.features.GeoJson(
                        data=GRP_levl2_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],    
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth6)
        ###############
    if key in GRP1:
        GRP_levl1_merge=pd.merge(levl1,GRP1[key],right_on="nuts1" ,left_on="NUTS_ID",how="right")

        choropleth7=folium.Choropleth(
        geo_data=GRP_levl1_merge,
        name="choropleth",
        data=GRP_levl1_merge,
        columns=["nuts1","PW DSV"],
        key_on="properties.NUTS_ID",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
        
        
        
                                    
        ).geojson.add_to(fg7)



        folium.features.GeoJson(
                        data=GRP_levl1_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],       
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth7) 
        #############
    if key in GRP0:
        GRP_levl0_merge=pd.merge(levl0,GRP0[key],right_on="nuts0" ,left_on="ISO2",how="right")

        choropleth8=folium.Choropleth(
        geo_data=GRP_levl0_merge,
        name="choropleth",
        data=GRP_levl0_merge,
        columns=["nuts0","PW DSV"],
        key_on="properties.ISO2",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
        
        
        
                                    
        ).geojson.add_to(fg8)



        folium.features.GeoJson(
                        data=GRP_levl0_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NAME","PW DSV"],
                                    
                                
                            aliases=["To : ", "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth8) 

        #################################################
        #FTL:
    
       
    if key in FTL3:
        FTL_levl3_merge=pd.merge(levl3,FTL3[key],right_on="NUTS3" ,left_on="NUTS_ID",how="right")

        choropleth9=folium.Choropleth(
        geo_data=FTL_levl3_merge,
        name="choropleth",
        data=FTL_levl3_merge,
        columns=["NUTS3","PW DSV"],
        key_on="properties.NUTS_ID",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
                       
        ).geojson.add_to(fg9)



        folium.features.GeoJson(
                        data=FTL_levl3_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],
                                    
                                
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth9) 
        ########################

        

    if key in FTL2:
        FTL_levl2_merge=pd.merge(levl2,FTL2[key],right_on="nuts2" ,left_on="NUTS_ID",how="right")

        choropleth10=folium.Choropleth(
            geo_data=FTL_levl2_merge,
            name="choropleth",
            data=FTL_levl2_merge,
            columns=["nuts2","PW DSV"],
            key_on="properties.NUTS_ID",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg10)



        folium.features.GeoJson(
                        data=FTL_levl2_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],    
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth10)
            ###############
    if key in FTL1:
        FTL_levl1_merge=pd.merge(levl1,FTL1[key],right_on="nuts1" ,left_on="NUTS_ID",how="right")

        choropleth11=folium.Choropleth(
            geo_data=FTL_levl1_merge,
            name="choropleth",
            data=FTL_levl1_merge,
            columns=["nuts1","PW DSV"],
            key_on="properties.NUTS_ID",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg11)



        folium.features.GeoJson(
                        data=FTL_levl1_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],       
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth11) 
            #############
    if key in FTL0:
        FTL_levl0_merge=pd.merge(levl0,FTL0[key],right_on="nuts0" ,left_on="ISO2",how="right")

        choropleth12=folium.Choropleth(
            geo_data=FTL_levl0_merge,
            name="choropleth",
            data=FTL_levl0_merge,
            columns=["nuts0","PW DSV"],
            key_on="properties.ISO2",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg12)



        folium.features.GeoJson(
                        data=FTL_levl0_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NAME","PW DSV"],
                                    
                                
                            aliases=["To : ", "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth12) 
        ########################################################
        #LTL :
    
       
    if key in LTL3:
        LTL_levl3_merge=pd.merge(levl3,LTL3[key],right_on="NUTS3" ,left_on="NUTS_ID",how="right")
    
        choropleth13=folium.Choropleth(
        geo_data=LTL_levl3_merge,
        name="choropleth",
        data=LTL_levl3_merge,
        columns=["NUTS3","PW DSV"],
        key_on="properties.NUTS_ID",
        fill_color='OrRd',
        fill_opacity=0.7,
        legend=True,
        highlight=True,
                       
        ).geojson.add_to(fg13)



        folium.features.GeoJson(
                        data=LTL_levl3_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],
                                    
                                
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth13) 
        ########################

        

    if key in LTL2:
        LTL_levl2_merge=pd.merge(levl2,LTL2[key],right_on="nuts2" ,left_on="NUTS_ID",how="right")

        choropleth14=folium.Choropleth(
            geo_data=LTL_levl2_merge,
            name="choropleth",
            data=LTL_levl2_merge,
            columns=["nuts2","PW DSV"],
            key_on="properties.NUTS_ID",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg14)



        folium.features.GeoJson(
                        data=LTL_levl2_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],    
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth14)
            ###############
    if key in LTL1:
        LTL_levl1_merge=pd.merge(levl1,LTL1[key],right_on="nuts1" ,left_on="NUTS_ID",how="right")

        choropleth15=folium.Choropleth(
            geo_data=LTL_levl1_merge,
            name="choropleth",
            data=LTL_levl1_merge,
            columns=["nuts1","PW DSV"],
            key_on="properties.NUTS_ID",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg15)



        folium.features.GeoJson(
                        data=LTL_levl1_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NUTS_NAME","ZC to","PW DSV"],       
                            aliases=["To : ","ZipCode  : " , "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth15) 
            #############
    if key in LTL0:
        LTL_levl0_merge=pd.merge(levl0,LTL0[key],right_on="nuts0" ,left_on="ISO2",how="right")

        choropleth16=folium.Choropleth(
            geo_data=LTL_levl0_merge,
            name="choropleth",
            data=LTL_levl0_merge,
            columns=["nuts0","PW DSV"],
            key_on="properties.ISO2",
            fill_color='OrRd',
            fill_opacity=0.7,
            legend=True,
            highlight=True,
            
            
            
                                        
            ).geojson.add_to(fg16)



        folium.features.GeoJson(
                        data=LTL_levl0_merge,
                        name='test',
                        smooth_factor=2,
                        style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                        tooltip=folium.features.GeoJsonTooltip(
                            fields=["NAME","PW DSV"],
                                    
                                
                            aliases=["To : ", "Pay Weight : "], 
                            localize=True,
                            sticky=False,
                            labels=True,
                            style="""
                                background-color: #F0EFEF;
                                border: 2px solid black;
                                border-radius: 3px;
                                box-shadow: 3px;
                            """,
                            max_width=800,),
                                highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                            ).add_to(choropleth16)
        


    #########
    folium.LayerControl().add_to(m)
    Draw(export=True).add_to(m)

    

    GroupedLayerControl(
        groups={'ALL data': [fg0,fg1, fg2,fg3]}, 
        exclusive_groups=False,     
        collapsed=False,).add_to(m)
    
    GroupedLayerControl(
        groups={'GRP data': [fg8,fg7, fg6,fg5]},  
        exclusive_groups=False,
        collapsed=False,).add_to(m)
    
    GroupedLayerControl(
        groups={'FTL data': [fg12,fg11, fg10,fg9]}, 
        exclusive_groups=False,     
        collapsed=False,).add_to(m)
    
    GroupedLayerControl(
        groups={'LTL data': [fg16,fg15, fg14,fg13]},
        exclusive_groups=False,      
        collapsed=False,).add_to(m)

    GroupedLayerControl(
    groups={'DSV Road': [fg4]}, 
    exclusive_groups=False,
    collapsed=False,).add_to(m)
    ###############

    categories=dsv["Country"].unique().tolist()
    for k in range(len(dsv)):

        location = dsv["lat"].iloc[k],dsv["lon"].iloc[k]
        html= f"<span style='font-family:Arial;''><span style='color:#002664;'><b>{ dsv['Office Name'].iloc[k]} <br> { dsv['ZC'].iloc[k]}  </b> </span><br> <br>  &#9743; {dsv['Phone'].iloc[k]} <br> <span style='font-size:12px'> &#128343; </span> {dsv['Opening hours'].iloc[k]} </span>"
        iframe = branca.element.IFrame(html=html, width=250, height=140)


        folium.Marker(
        location=location,
        tags=[dsv["Country"].iloc[k]],
        icon=folium.Icon(color='darkblue',icon_color='White', icon="info-sign"),
        tooltip=dsv["Office Name"].iloc[k] + " , click for more information",
        popup = folium.Popup(iframe, max_width=500)
        
        ).add_to(fg4)

        
    TagFilterButton(categories).add_to(m)

    m.save(f"{key}_map.html")

    